In [ ]:
import logging
import requests
import csv
import os
import psycopg2

from datetime import datetime
from telegram import ForceReply, Update
from telegram.ext import Application, CommandHandler, ContextTypes, MessageHandler, filters
from telegram.ext import CallbackContext

data = [('user_id', 'datetime', 'action')]

#создаем файл csv если он еще не существует и создаем шапку таблички
if not os.path.exists('user_events.csv'):
    with open('user_events.csv', 'w', newline='') as f:
        csv.writer(f).writerows(data)
    
OAUTH_TOKEN = "y0__xDlwfykqveAAhjB3RMg_6Pl4RL7NW44PHqgGnaDuSITbqi_d7qxuA"

def get_iam_token():
    response = requests.post(
        'https://iam.api.cloud.yandex.net/iam/v1/tokens',
        json={'yandexPassportOauthToken': OAUTH_TOKEN}
    )

    response.raise_for_status()
    return response.json()['iamToken']

# Enable logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO
)
# set higher logging level for httpx to avoid all GET and POST requests being logged
logging.getLogger("httpx").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)

#создаем соединение с бд на виртуальной машине
connection = psycopg2.connect(dbname="yaga_bot_db", user="telegram_bot_user", password="strong_password", host="localhost")

def track_user_action(user_id, event_time, action_text, filename="user_events.csv"):
    """Записываем каждую активность в csv файл"""
    with open(filename, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow([user_id, event_time.isoformat(), action_text])
        
def db_write_user_action(user_id, event_time, action_text, connection):
    """Записываем каждую активность в базу данных на виртуальной машине"""
    cursor = connection.cursor()
    cursor.execute("INSERT INTO user_actions (user_id, event_time, action) VALUES (%s, %s, %s)",
    (user_id, event_time, action_text))
    connection.commit()
    cursor.close()

# Define a few command handlers. These usually take the two arguments update and
# context.
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    user_id = update.effective_user.id
    event_time = update.message.date
    track_user_action(user_id, event_time, "start")
    connection = context.application.bot_data["db_connection"]
    db_write_user_action(user_id, event_time, "start", connection)
    await update.message.reply_html(
        rf"Hi {user.mention_html()}!",
        reply_markup=ForceReply(selective=True),
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /help is issued."""
    user_id = update.effective_user.id
    event_time = update.message.date
    track_user_action(user_id, event_time, "help")
    connection = context.application.bot_data["db_connection"]
    db_write_user_action(user_id, event_time, "help", connection)
    await update.message.reply_text("Help!")
    
async def process_message(update: Update, context: CallbackContext) -> None:
    user_text = update.message.text
    user_id = update.effective_user.id
    event_time = update.message.date
    track_user_action(user_id, event_time, "answer")
    connection = context.application.bot_data["db_connection"]
    db_write_user_action(user_id, event_time, "answer", connection)
    # Получаем IAM токен
    iam_token = get_iam_token()

    # Собираем запрос
    data = {}
    # Указываем тип модели
    data["modelUri"] = f"gpt://b1g3a2v49k0higqru6cq/yandexgpt/latest"
    # Настраиваем опции
    data["completionOptions"] = {"temperature": 0.3, "maxTokens": 1000}
    # Указываем контекст для модели
    data["messages"] = [
        {"role": "user", "text": f"{user_text}"},
    ]

    URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    # Отправляем запрос
    response = requests.post(
        URL,
        headers={
            "Accept": "application/json",
            "Authorization": f"Bearer {iam_token}"
        },
        json=data,
    ).json()

    answer = response.get('result', {})\
                     .get('alternatives', [{}])[0]\
                     .get('message', {})\
                     .get('text', {})

    await update.message.reply_text(answer)


def main() -> None:
    """Start the bot."""
    # Create the Application and pass it your bot's token.
    application = Application.builder().token("7957784669:AAFtF3MlDV_hfE2cgaAtZCW29weZrm7WIIQ").build()
    application.bot_data["db_connection"] = connection

    # on different commands - answer in Telegram
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, process_message))

    # Run the bot until the user presses Ctrl-C
    application.run_polling(allowed_updates=Update.ALL_TYPES)


if __name__ == "__main__":
    main()